In [2]:
import pandas as pd

In [3]:
data_transactions = {
    'transaction_id': [1, 2, 3, 4, 5, 6, 7, 8],
    'client_id': [101, 102, 101, 103, 102, 101, 103, 102],
    'transaction_date': ['2024-01-15', '2024-02-20', '2024-03-10', '2024-04-05', '2024-05-12', '2023-12-25', '2024-06-01', '2024-07-18'],
    'amount': ['1000.50', '200.75', '150.00', '500.25', '300.50', '1200.00', '700.00', '400.00'],
    'currency': ['USD', 'EUR', 'USD', 'USD', 'EUR', 'USD', 'USD', 'EUR'],
    'transaction_type': ['deposit', 'withdrawal', 'deposit', 'withdrawal', 'deposit', 'deposit', 'withdrawal', 'withdrawal']
}

In [4]:
data_clients = {
    'client_id': [101, 102, 103],
    'client_name': ['Иван Иванов', 'Петр Петров', 'Сидор Сидоров'],
    'registration_date': ['2023-12-01', '2024-01-10', '2024-02-15'],
    'client_status': ['active', 'active', 'inactive']
}

In [5]:
transactions = pd.DataFrame(data_transactions)

In [6]:
clients = pd.DataFrame(data_clients)

In [7]:
transactions

,transaction_id,client_id,transaction_date,amount,currency,transaction_type
0,1,101,2024-01-15,1000.50,USD,deposit
1,2,102,2024-02-20,200.75,EUR,withdrawal
2,3,101,2024-03-10,150.00,USD,deposit
3,4,103,2024-04-05,500.25,USD,withdrawal
4,5,102,2024-05-12,300.50,EUR,deposit
5,6,101,2023-12-25,1200.00,USD,deposit
6,7,103,2024-06-01,700.00,USD,withdrawal
7,8,102,2024-07-18,400.00,EUR,withdrawal


In [8]:
clients

,client_id,client_name,registration_date,client_status
0,101,Иван Иванов,2023-12-01,active
1,102,Петр Петров,2024-01-10,active
2,103,Сидор Сидоров,2024-02-15,inactive


In [9]:
transactions["amount"] = pd.to_numeric(transactions["amount"])


In [10]:
inactive_clients = clients.loc[clients["client_status"] != "active","client_id"]
inactive_clients

2    103
Name: client_id, dtype: int64

In [11]:
transactions = transactions.drop(transactions[transactions["client_id"].isin(inactive_clients)].index)
transactions

,transaction_id,client_id,transaction_date,amount,currency,transaction_type
0,1,101,2024-01-15,1000.50,USD,deposit
1,2,102,2024-02-20,200.75,EUR,withdrawal
2,3,101,2024-03-10,150.00,USD,deposit
4,5,102,2024-05-12,300.50,EUR,deposit
5,6,101,2023-12-25,1200.00,USD,deposit
7,8,102,2024-07-18,400.00,EUR,withdrawal


In [12]:
clients_transactions = clients[clients["client_status"] == "active"].copy()
clients_transactions = clients_transactions.drop(columns=["registration_date", "client_status"])
clients_transactions


,client_id,client_name
0,101,Иван Иванов
1,102,Петр Петров


In [13]:

total_amount = transactions.groupby(["client_id","transaction_type"])["amount"].sum().rename("total_amount").reset_index().sort_values(by="total_amount",ascending=False)
clients_transactions = pd.merge(clients_transactions,total_amount)
total_amount


,client_id,transaction_type,total_amount
0,101,deposit,2350.50
2,102,withdrawal,600.75
1,102,deposit,300.50


In [14]:
average_amount = transactions.groupby(["client_id","transaction_type"])["amount"].mean().rename("average_amount").reset_index()
clients_transactions = pd.merge(clients_transactions,average_amount)
average_amount

,client_id,transaction_type,average_amount
0,101,deposit,783.500
1,102,deposit,300.500
2,102,withdrawal,300.375


In [15]:
# Количество транзакций по типам операций
transaction_count = transactions.groupby(["client_id","transaction_type"])["amount"].count().rename("transaction_count").reset_index()
clients_transactions = pd.merge(clients_transactions,transaction_count)
transaction_count

,client_id,transaction_type,transaction_count
0,101,deposit,3
1,102,deposit,1
2,102,withdrawal,2


In [16]:
# Дата последней транзакции
last_transaction_date = transactions.groupby("client_id")["transaction_date"].max().rename("last_transaction_date").reset_index()
clients_transactions = pd.merge(clients_transactions,last_transaction_date)
last_transaction_date

,client_id,last_transaction_date
0,101,2024-03-10
1,102,2024-07-18


In [17]:
clients_transactions

,client_id,client_name,transaction_type,total_amount,average_amount,transaction_count,last_transaction_date
0,101,Иван Иванов,deposit,2350.50,783.500,3,2024-03-10
1,102,Петр Петров,withdrawal,600.75,300.375,2,2024-07-18
2,102,Петр Петров,deposit,300.50,300.500,1,2024-07-18
